# Developing maximally-symmetric lattice population algorithm

In [ ]:
from polymerist.graphics.plotutils import presize_subplots

max_n_atoms : int = 10_000

records = {}
for mol_name, n_atoms in counts.items():
    n_oligomers = max_n_atoms // n_atoms
    cubic_lattice_dim = np.ceil(n_oligomers**(1/3))

    lattice_info = {
        'n_atoms'  : n_atoms,
        'n_oligomers' : n_oligomers,
        'occupancy' : n_oligomers / cubic_lattice_dim**3,
        'vacancy' : cubic_lattice_dim**3 - n_oligomers,
    }
    records[mol_name] = lattice_info

df = pd.DataFrame.from_dict(records, orient='index')

fig, ax = presize_subplots(nrows=2, ncols=2, scale=10, elongation=1.0)
for colname, axis in zip(df.columns, ax.flatten()):
    heights, bins, patches = axis.hist(df[colname])
    axis.set_xlabel(colname)
    axis.set_ylabel('Counts')

plt.show()

In [ ]:
import numpy as np
from rdkit import Chem
from rdkit.Chem.AllChem import EmbedMolecule
from polymerist.monomers import specification

smi = '[Br]-[C]12-[C]3(-[I])-[C]4(-[Cl])-[C]-1(-[N])-[C]1(-[I])-[C]-2(-[Cl])-[C]-3(-[N])-[C]-4-1-[Br]'
print(smi)
mol = Chem.MolFromSmiles(smi, sanitize=True)
mol = Chem.AddHs(mol)
EmbedMolecule(mol)
display(mol)

Chem.MolToMolFile(mol, 'cubane.mol')

# Playing with lattice generation (i.e. generating non-cubic lattices)

In [ ]:
from rdkit import Chem
from rdkit.Chem.AllChem import EmbedMolecule
from rdkit.Chem import rdMolTransforms

In [1]:
from typing import Annotated, Any, Literal, Optional, TypeAlias, TypeVar
import numpy as np
import numpy.typing as npt


N = TypeVar('N', bound=int) # for typing arbitrary array dimension
M = TypeVar('M', bound=int) # for typing arbitrary array dimension
DType = TypeVar('DType', bound=np.generic)
Shape : TypeAlias = tuple

Coords2D : TypeAlias = np.ndarray[Shape[M, 2], DType] # an Mx2 array of M 3D coordinates
Coords3D : TypeAlias = np.ndarray[Shape[M, 3], DType] # an Mx3 array of M 3D coordinates
CoordsND : TypeAlias = np.ndarray[Shape[M, N], DType] # an MxN array of M ND coordinates

Vector2D : TypeAlias = np.ndarray[Shape[2], DType] # a 1x2 array of M 3D coordinates
Vector3D : TypeAlias = np.ndarray[Shape[3], DType] # a 1x3 array of M 3D coordinates
VectorND : TypeAlias = np.ndarray[Shape[N], DType] # a 1xN array of M ND coordinates



T = TypeVar('T')
def inv_left(matrix : np.ndarray[Shape[M, N], DType]) -> np.ndarray[Shape[N, M], DType]:
    '''Return the left-inverse of a non-square matrix'''
    return np.linalg.inv(matrix.T @ matrix) @ matrix.T

def inv_right(matrix : np.ndarray[Shape[M, N], DType]) -> np.ndarray[Shape[N, M], DType]:
    '''Return the right-inverse of a non-square matrix'''
    return matrix.T @ np.linalg.inv(matrix @ matrix.T)

from typing import Any
Shape : TypeAlias = tuple


def mean_coord(coords : np.ndarray[Shape[M, N], DType]) -> np.ndarray[Shape[N], DType]:
    '''Compute the average (center-of-mass) coordinate of a vector of coordinates'''
    return coords.mean(axis=0)

def dists_to_point(coords : np.ndarray[Shape[M, N], DType], point : np.ndarray[Shape[N], DType], norm_order : Optional[int]=None) -> np.ndarray[Shape[M], DType]:
    '''Compute the distance between each point in a coordinate array and a single arbitrary point'''
    return np.linalg.norm(coords - point, ord=norm_order, axis=1)

def dists_to_centroid(coords : np.ndarray[Shape[M, N], DType], norm_order : Optional[int]=None) -> np.ndarray[Shape[M], DType]:
    '''Compute the distance of each coordinate in an array of coordinates to the coordinates' centroid'''
    return dists_to_point(coords, point=mean_coord(coords), norm_order=norm_order)


def bounding_box(coords : np.ndarray[Shape[M, N], DType]) -> np.ndarray[Shape[2, N], DType]:
    '''Compute the tight bounding box around an array of coordinates'''
    return np.vstack([coords.min(axis=0), coords.max(axis=0)])

def bounding_box_dims(coords : np.ndarray[Shape[M, N], DType]) -> np.ndarray[Shape[N], DType]:
    '''Compute the tight bounding box dimensions around an array of coordinates'''
    return coords.ptp(axis=0)


def lex_order_coords(coords : np.ndarray[Shape[M, N]]) -> np.ndarray[Shape[M], int]:
    '''determine the lexicographic ordering of a coordinate array'''
    return np.lexsort(coords.T)

def odd_even_lattice(coords : np.ndarray[Shape[M, N]]) -> tuple[np.ndarray[Shape[M, N]], np.ndarray[Shape[M, N]]]:
    '''Partition a lattice into odd and even indexed sublattices, returned as two coordinate arrays
    Sublattices have the property that an odd points 1-connected nearest neighbors are all even, and vice-versa'''
    pass

In [143]:
from dataclasses import dataclass

@dataclass
class LatticeParameters:
    '''For storing the lengths of and the angles between the 3 lattice vectors of a crystallographic unit cell'''
    a : float
    b : float
    c : float

    alpha : float
    beta  : float
    gamma : float

    def __post_init__(self) -> None:
        '''Validating initialized values'''
        for axial_length in self.axial_lengths:
            assert(axial_length > 0) # ensure vector lengths are positive

        for angle_attr in ('alpha', 'beta', 'gamma'):
            angle = getattr(self, angle_attr)
            setattr(self, angle_attr, angle % (2*np.pi)) # ensure angles are within [0, 2*pi)

    # LATTICE VECTOR METHODS
    @classmethod
    def from_lattice_vectors(cls, vectors : np.ndarray[Shape[3,3], float]) -> 'LatticeParameters':
        '''Obtain axial lengths and inter-vector angles from a matrix of lattice vectors'''
        axial_lengths = np.linalg.norm(vectors, axis=1)
        vectors = vectors / axial_lengths[:, None] # normalize along rows
        print(vectors, np.linalg.norm(vectors, axis=1))

        cycled_dots = np.sum(np.roll(vectors, 1, axis=0) * np.roll(vectors, -1, axis=0), axis=1) # row-wise dot product between cycles rows, giving [vectors.C, C.A, and A.vectors] as result
        print(cycled_dots)
        angles = np.arccos(cycled_dots) # invert cosine from dot product expression to recover angles

        return cls(*axial_lengths, *angles)

    def to_lattice_vectors(self) -> np.ndarray[Shape[3,3], float]:
        '''
        The restricted lattice vectors corresponding to the lattice parameters,
        where vector A lies along the x-axis and vector B is in the xy-plane
        
        Vectors are returned as a 3x3 matrix, where each row represents a lattice vector
        '''
        # formula taken from https://www.aflowlib.org/prototype-encyclopedia/triclinic_lattice.html 
        ax = 1.0
        bx = np.cos(self.gamma)
        by = np.sin(self.gamma)
        cx = np.cos(self.beta)
        cy = (np.cos(self.alpha) - np.cos(self.beta)*np.cos(self.gamma)) / np.sin(self.gamma)
        cz = np.sqrt(1.0 - cx**2 - cy**2)

        unit_matr = np.array([
            [ax, 0.0, 0.0],
            [bx, by, 0.0],
            [cx, cy, cz],
        ]).astype(np.float64)
        return unit_matr @ np.diag(self.lengths).astype(np.float64) # order matters here!

    @property
    def lattice_vectors(self) ->np.ndarray[Shape[3,3], float]:
        '''Property alias of to_lattice_vectors() method for convenience'''
        return self.to_lattice_vectors()

    def volume(self) -> float:
        '''The volume of the unit cell (in arbitrary units)'''
        return np.linalg.det(self.lattice_vectors)
    
    # PROPERTY VIEWS
    @property
    def axial_lengths(self) -> np.ndarray[Shape[3], float]:
        '''View of just the axial lengths'''
        return np.array([self.a, self.b, self.c])
    lengths = axial_lengths

    def axial_angles(self, in_degrees : bool=False) -> np.ndarray[Shape[3], float]:
        '''View of just the angles'''
        angles = np.array([self.alpha, self.beta, self.gamma])
        if in_degrees:
            angles = np.rad2deg(angles)
        return angles
    
    @property
    def angles(self) -> np.ndarray[Shape[3], float]:
        '''Property alias of angles() method for convenience'''
        return self.axial_angles(in_degrees=False)

In [144]:
TO = np.array([
    [1, 0, 0],
    [1/3, np.sqrt(2)*2/3, 0],
    [-1/3, np.sqrt(2)/3, np.sqrt(6)/3],
])
RDC = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [1/2, 1/2, 1/np.sqrt(2)],
])
TO

array([[ 1.        ,  0.        ,  0.        ],
       [ 0.33333333,  0.94280904,  0.        ],
       [-0.33333333,  0.47140452,  0.81649658]])

In [149]:
lens = np.array([8.508, 11.185, 7.3], dtype=np.float64)
angs = np.deg2rad(np.array([90.85, 114.1, 80.0], dtype=np.float64))
lp = LatticeParameters(*lens, *angs)
lp

LatticeParameters(a=8.508, b=11.185, c=7.3, alpha=1.5856316254368483, beta=1.99142067652553, gamma=1.3962634015954636)

In [150]:
matr = lp.lattice_vectors
matr

array([[ 8.508     ,  0.        ,  0.        ],
       [ 1.4773987 , 11.01507472,  0.        ],
       [-3.47407556,  0.63682997,  6.65071474]])

In [151]:
lp2 = LatticeParameters.from_lattice_vectors(matr)
lp2

[[ 1.          0.          0.        ]
 [ 0.13293477  0.99112479  0.        ]
 [-0.46134078  0.084568    0.88318341]] [1. 1. 1.]
[ 0.02248921 -0.46134078  0.13293477]


LatticeParameters(a=8.508, b=11.113711258465901, c=7.530389099802367, alpha=1.5483052178059842, beta=2.050302138657309, gamma=1.4374668764441623)

In [152]:
lp2.axial_angles(in_degrees=True)

array([ 88.71135438, 117.47365927,  82.36078521])

In [ ]:
def 

In [70]:
ang2 = []
for rowpair in ((1,2), (0,2), (0,1)):
    row1, row2 = matr[rowpair, :]
    angle = np.dot(row1, row2)
    ang2.append(angle)

In [74]:
np.arccos(np.clip(ang2, -1, 1))

array([0.        , 3.14159265, 0.        ])

In [66]:
row1, row2

(array([8.508, 0.   , 0.   ]), array([ 1.4773987 , 11.01507472,  0.        ]))

In [63]:
lp2 = LatticeParameters.from_lattice_vectors(matr)
lp2

/tmp/ipykernel_227791/901799723.py:31: RuntimeWarning: invalid value encountered in arccos
  angles = np.arccos(cycled_dots) # invert cosine from dot product expression to recover angles


LatticeParameters(a=8.508, b=11.113711258465901, c=7.530389099802367, alpha=nan, beta=nan, gamma=nan)

In [39]:
# A = np.random.rand(3,3) * np.random.rand(3)
A = RDC
A = TO
# A = 10.0 * np.eye(3)
A[0,1] = A[0,2] = A[1,2] = 0
print(A)

lp = LatticeParameters.from_lattice_vectors(A)
lp

[[ 1.          0.          0.        ]
 [ 0.33333333  0.94280904  0.        ]
 [-0.33333333  0.47140452  0.81649658]]


LatticeParameters(a=1.0, b=1.0, c=0.9999999999999999, alpha=3.8671730623849494, beta=6.002431338704408, gamma=3.8671730623849503)

In [34]:
lp.axial_angles(in_degrees=True)

array([ 70.52877937, 109.47122063,  70.52877937])

In [35]:
lp.lattice_vectors, A

(array([[ 1.        ,  0.        ,  0.        ],
        [ 0.33333333,  0.94280904,  0.        ],
        [-0.33333333,  0.47140452,  0.81649658]]),
 array([[ 1.        ,  0.        ,  0.        ],
        [ 0.33333333,  0.94280904,  0.        ],
        [-0.33333333,  0.47140452,  0.81649658]]))

In [36]:
(lp.lattice_vectors - A).astype(np.float64)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-5.55111512e-17, -1.11022302e-16,  0.00000000e+00],
       [ 1.66533454e-16,  5.55111512e-17,  0.00000000e+00]])

In [ ]:
np.sin(lp.angles), np.cos(lp.angles)

In [ ]:
latt = generate_int_lattice(2,2,2)
order = lex_order_coords(latt)
latt = latt[order]

nest = latt.reshape(2,2,2,3)
nest.reshape(-1, nest.shape[-1])

In [ ]:
theta = np.pi / 3#np.arccos(1/2)
theta

# A = np.array([
#     [1, np.cos(theta)],
#     [0, np.sin(theta)],
# ])
A = np.array([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [1/2, 1/2, 1/np.sqrt(2)],
])
sheared = latt @ TO.T

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(*sheared.T)

In [ ]:
A_inv = np.linalg.inv(A)
inv_left(A) - A_inv, inv_right(A) - A_inv

In [ ]:
np.indices(*latt)

In [ ]:
from polymerist.monomers import specification

smi = 'CC(=O)OC'
smi = specification.expanded_SMILES(smi, assign_map_nums=False)
rdmol = Chem.MolFromSmiles(smi, sanitize=False)
display(rdmol)

In [ ]:
err = EmbedMolecule(rdmol)
assert(err == 0)
display(rdmol)

In [ ]:
conf = rdmol.GetConformer(0)
pos  = conf.GetPositions() 
pos

# Playing with cubic/octahedral symmetry groups

In [ ]:
import numpy as np
from polymerist.graphics.plotutils import scatter_3D
from polymerist.maths.lattices import generate_int_lattice

d : int = 4

# produce integer lattice
dims = np.array([d, d, d])
num_points = dims.prod()
lattice = generate_int_lattice(*dims)

# sort lexicographically
order = np.lexsort(lattice.T)
lattice = lattice[order]

# determine even and odd positions after sorting
index_is_odd = lattice.sum(axis=1) % 2
index_is_odd = index_is_odd.astype(bool)

# center lattice at the origin
COM = lattice.mean(axis=0)
lattice = lattice - COM # sort lexicographically and translate center of grid to origin

# print(order, lattice[order])

In [ ]:
import matplotlib.pyplot as plt

alternate : bool = not False
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

if alternate:
    ax.scatter(*lattice[ index_is_odd].T, color='r')
    ax.scatter(*lattice[~index_is_odd].T, color='b')
else:
    ax.scatter(*lattice.T, color='r')

labels = np.lexsort(lattice.T)
for i, coords in zip(labels, lattice):
    ax.text(*coords, str(i))

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

plt.show()

In [ ]:
from typing import Generator, Iterable
from itertools import permutations, product as cartesian_product
from polymerist.maths.combinatorics.permutations import Permutation, Cycle
from scipy.spatial.transform import Rotation, Slerp


def sign_alternations(n : int) -> Generator[tuple[int, int, int], None, None]:
    '''Generate every possible n-tuple containing either 1 or -1'''
    for signs in cartesian_product(*([1, -1] for _ in range(n))):
        yield signs

def orthogonal_basis_transforms(n : int=3) -> Generator[np.ndarray, None, None]:
    '''Generate all matrices in n-dimensions which permute or invert the standard basis vectors'''
    for signs in sign_alternations(n):
        matr = np.diag(signs)
        for perm in Permutation.symmetric_group(n):
            yield matr @ perm.matrix

def interpolate_linear_transformation(trans : np.ndarray, n_frames : int=10) -> np.ndarray:
    '''Accepts a linear transformation matrix (of size MxN) and a number of frames F
    Returns an FxMxN array containing uniforms "steps" between the identity and the desired transformation'''
    I = Rotation.identity()
    rot = Rotation.from_matrix(trans)
    full_rot = Rotation.concatenate([I, rot])
    interpolator = Slerp(np.linspace(0, 1, num=len(full_rot), dtype=int), full_rot)

    return interpolator(np.linspace(0, 1, num=n_frames)).as_matrix()

In [ ]:
from collections import defaultdict

perms  = {}
all_cycles = {}

orbits = defaultdict(set)
stabilizers = defaultdict(list)

init_order = np.lexsort(lattice.T)
group_matrices = [el for el in orthogonal_basis_transforms(3)]
for i, trans in enumerate(group_matrices):
    new = lattice @ trans
    perms[i]      = perm   = Permutation(*np.lexsort(new.T))
    all_cycles[i] = cycles = perm.to_cycles(canonicalize=False)
    
    for cycle in cycles:
        if len(cycle) == 1:
            point_idx = cycle[0] # get the one element in the singleton cycle
            stabilizers[point_idx].append(trans)

        for point_idx in cycle:
            orbits[point_idx].update(cycle)

unique_orbits = set(tuple(sorted(orbit)) for orbit in orbits.values())
sum(len(i) for i in unique_orbits) == num_points

In [ ]:
stab_sizes = {k : len(v) for k, v in stabilizers.items()}
for i in unique_orbits:
    print([stab_sizes[j] for j in i])

## Visualize all symmetry actions as GIFs

In [ ]:
from pathlib import Path
from math import ceil
from matplotlib.animation import FuncAnimation, ArtistAnimation, writers
from rich.progress import track


n_frames = 20

dir = Path('gif_frames')
dir.mkdir(exist_ok=True)

ax_max = ceil(d / 2)
ax_min = -ax_max

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.autoscale(False)

def update(frame_no : int):
    partial_rot = keyframes[frame_no, :, :]
    latt_pos = lattice @ partial_rot

    ax.clear()
    ax.set(xlim3d=(ax_min, ax_max), xlabel='X')
    ax.set(ylim3d=(ax_min, ax_max), ylabel='Y')
    ax.set(zlim3d=(ax_min, ax_max), zlabel='Z')

    return ax.scatter(*latt_pos.T)

for i, trans in track(enumerate(group_matrices), total=len(group_matrices), description='Visualizing symmetry group actions...'):
    keyframes = interpolate_linear_transformation(trans, n_frames=n_frames)
    ani = FuncAnimation(fig=fig, func=update, frames=len(keyframes), interval=10)
    ani.save(dir/f'symmetry_{i+1}.gif', writer='pillow')
    plt.close(fig)

# Playing with rich progress

In [ ]:
from rich.progress import track, Progress
from time import sleep

with Progress() as progress:

    task1 = progress.add_task("[red]Downloading...", total=1000)
    task2 = progress.add_task("[green]Processing...", total=1000)
    task3 = progress.add_task("[cyan]Cooking...", total=1000)

    while not progress.finished:
        progress.update(task1, advance=0.5)
        progress.update(task2, advance=0.3)
        progress.update(task3, advance=0.9)
        sleep(0.02)

In [ ]:
import time
import random
from rich.progress import (
    BarColumn,
    Progress,
    SpinnerColumn,
    TaskProgressColumn,
    TimeElapsedColumn,
    TimeRemainingColumn,
)

def process(chunks):
    for chunk in chunks:
        time.sleep(0.1)
        yield chunk

chunks = [random.randint(1,20) for _ in range(100)]

progress_columns = (
    SpinnerColumn(),
    "[progress.description]{task.description}",
    BarColumn(),
    TaskProgressColumn(),
    "Elapsed:",
    TimeElapsedColumn(),
    "Remaining:",
    TimeRemainingColumn(),
)

with Progress(*progress_columns) as progress_bar:
    task = progress_bar.add_task("[blue]Downloading...", total=sum(chunks))
    for chunk in process(chunks):
        progress_bar.update(task, advance=chunk)

In [ ]:
import random
import time

from rich.live import Live
from rich.table import Table


def generate_table() -> Table:
    """Make a new table."""
    table = Table()
    table.add_column("ID")
    table.add_column("Value")
    table.add_column("Status")

    for row in range(random.randint(2, 6)):
        value = random.random() * 100
        table.add_row(
            f"{row}", f"{value:3.2f}", "[red]ERROR" if value < 50 else "[green]SUCCESS"
        )
    return table


with Live(generate_table(), refresh_per_second=4) as live:
    for _ in range(40):
        time.sleep(0.4)
        live.update(generate_table())

In [ ]:
from dataclasses import dataclass
from rich.console import Console, ConsoleOptions, RenderResult
from rich.table import Table

@dataclass
class Student:
    id: int
    name: str
    age: int
    def __rich_console__(self, console: Console, options: ConsoleOptions) -> RenderResult:
        yield f"[b]Student:[/b] #{self.id}"
        my_table = Table("Attribute", "Value")
        my_table.add_row("name", self.name)
        my_table.add_row("age", str(self.age))
        yield my_table